# Process metadata CSV

In [1]:
import pandas as pd
import numpy as np
#import umap.umap_ as umap
import joblib
import torchvision.transforms as T
from LightningVAE.config import config
from LightningVAE.src.utils.model_factory import get_model
from LightningVAE.src.models.hyperparameters import params
from itertools import tee

/home/michael/anaconda3/envs/iml/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'src'

In [7]:
df = pd.read_csv("backend-project/data/HAM10000_metadata.csv")

---
## Populate with latent space coordinates

In [ ]:
input_dim = params[config['model']]['input_dim']
batch_size = params[config['model']]['batch_size']
transform = T.Compose([T.Resize((input_dim, input_dim)), T.ToTensor()])
model_path = '/home/jimmy/Medical1-xai-iml22/LightningVAE/reports/logs/20220513-033014_BetaVAEConv/epoch=61-step=9734.ckpt'
pretrained_model = get_model(params[config['model']], config['model'], f'')
pretrained_model = pretrained_model.load_from_checkpoint(model_path)


In [ ]:
file_names = df['image_id'].tolist()
file_names = [f'{name}.jpg' for name in file_names]

In [8]:
from PIL import Image
import re
import os
from torch.utils.data import Dataset

#TODO
file_names = df['image_id'].tolist()
file_names = [f'{name}.jpg' for name in file_names]

class XRayDataset(Dataset):
    '''
    Class to represent our dataset of XRay images. Subclass of torch.utils.data.Dataset. Store the image directory,
    images name, bounding boxes and data of the images we will use either for training or testing.
    '''

    def __init__(self, img_dir, images_name, transform=None):
        self.img_dir = img_dir
        # Read all the names of the PNG files in the directory
        images_dir = np.array([name for name in os.listdir(img_dir) if
                                os.path.isfile(os.path.join(img_dir, name)) and
                                (re.search('png$', name) is not None or re.search('jpg$', name) is not None)])
        print(len(images_name))
        self.images_name = images_name
        self.transform = transform
    def __len__(self):
        '''
        Overload: return the length of the dataset

        Return
        ----------
        length: înt:
            The length of the dataset
        '''
        return self.images_name.shape[0]

    def __getitem__(self, idx):

        img_path = os.path.join(self.img_dir, self.images_name[idx])
        img = Image.open(img_path)
        if self.transform:
            image = self.transform(img)
        return image

In [ ]:
DataLoader(self.train_set, batch_size=self.batch_size, shuffle=False)

---
## Populate with UMAP coordinates, and save UMAP model

In [9]:
latent_coordinates = df[['latent_coordinate' + str(i) for i in range(12)]]

In [10]:
# Fit UMAP model
reducer = umap.UMAP(random_state=0)
reducer = reducer.fit(latent_coordinates)

In [11]:
# Serialize UMAP model
joblib.dump(reducer, "umap.sav")

['umap.sav']

In [13]:
# Get UMAP coordinates
embeddings = reducer.transform(latent_coordinates)
df["umap1"] = [embedding[0] for embedding in embeddings]
df["umap2"] = [embedding[1] for embedding in embeddings]

---
## Save CSV

In [14]:
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,latent_coordinate0,latent_coordinate1,latent_coordinate2,...,latent_coordinate4,latent_coordinate5,latent_coordinate6,latent_coordinate7,latent_coordinate8,latent_coordinate9,latent_coordinate10,latent_coordinate11,umap1,umap2
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,183,847,915,...,491,-889,591,-93,-347,231,903,855,6.336992,3.048053
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,920,-710,-409,...,251,-92,-938,-343,-36,-665,-299,139,2.735142,3.801311
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,847,238,638,...,982,-960,433,956,-353,994,-835,944,6.763124,3.281678
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,395,283,29,...,-844,96,-530,-807,-823,-981,888,899,1.365654,4.038083
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,313,46,-104,...,523,579,-188,487,-212,-956,961,-764,5.678465,5.230410


In [15]:
df.to_csv("HAM10000_metadata_processed.csv", index=False)